In [1]:
system_prompt = """You are an AI Game Assistant. \
Your job is to detect changes to a player's \
inventory based on the most recent story and game state.
If a player picks up, or gains an item add it to the inventory \
with a positive change_amount.
If a player loses an item remove it from their inventory \
with a negative change_amount.
Given a player name, inventory and story, return a list of json update
of the player's inventory in the following form.
Only take items that it's clear the player (you) lost.
Only give items that it's clear the player gained. 
Don't make any other item updates.
If no items were changed return {"itemUpdates": []}
and nothing else.

Response must be in Valid JSON
Don't add items that were already added in the inventory

Inventory Updates:
{
    "itemUpdates": [
        {"name": <ITEM NAME>, 
        "change_amount": <CHANGE AMOUNT>}...
    ]
}
"""

In [3]:
import json
from helper import get_together_api_key, load_env
from together import Together

client = Together(api_key=get_together_api_key())

In [5]:
def detect_inventory_changes(game_state, output):
    inventory = game_state['inventory']
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": 
         f'Current Inventory: {str(inventory)}'},
        
        {"role": "user", "content": f'Recent Story: {output}'},
        {"role": "user", "content": 'Inventory Updates'}
    ]
    
    chat_completion = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        temperature=0.0,
        messages=messages
    )
    response = chat_completion.choices[0].message.content
    result = json.loads(response)
    return result['itemUpdates']

In [7]:
from helper import get_game_state

game_state = get_game_state()
game_state['inventory'] = {
    "cloth pants":1,
    "cloth shirt":1,
    "gold":5
}

result = detect_inventory_changes(game_state,"you buy a swaord from a merchant for 5 gold")

print(result)

[{'name': 'sword', 'change_amount': 1}, {'name': 'gold', 'change_amount': -5}]
